# Iteration 4

## Data Import

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/09 02:50:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
df = spark.read.csv('_Data/general_data.csv')

## Data Exploration

In [17]:
df.show()

+----------+---+---------+-----------------+--------------------+----------------+---------+--------------+-------------+------+--------+--------------------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+--------------------+--------------+--------------------+--------------------+--------------------+---------------+---------------+--------------+-----------------+
|       _c0|_c1|      _c2|              _c3|                 _c4|             _c5|      _c6|           _c7|          _c8|   _c9|    _c10|                _c11|         _c12|         _c13|              _c14|  _c15|             _c16|         _c17|            _c18|             _c19|                _c20|          _c21|                _c22|                _c23|                _c24|           _c25|           _c26|          _c27|             _c28|
+----------+---+---------+-----------------+--------------------+----------------+---------+--------------+-----

In [18]:
df.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28']

In [19]:
df.describe().show()

+-------+-----------------+------------------+---------+--------------+----------+----------------+------------------+----------------+-------------+------+------------------+--------------------+--------+-----------------+------------------+------+------------------+-------------+------------------+------------------+--------------------+------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|              _c0|               _c1|      _c2|           _c3|       _c4|             _c5|               _c6|             _c7|          _c8|   _c9|              _c10|                _c11|    _c12|             _c13|              _c14|  _c15|              _c16|         _c17|              _c18|              _c19|                _c20|              _c21|                _c22|                _c23|              _c24|              _c25|              _c26|              _c27|               _c28|
+-